In [ ]:
import os
import pandas as pd
import numpy as np
import argparse
import ast
from together import Together
from alive_progress import alive_bar

from prompt import *
from classes.utils import extract_option

client = Together(api_key="de1d1c231987694e2b9111e06e048732d206ecaee729b8aee41e2121006f2cfc")
PATH = "/home/cc/PHD/HealthBranches/"

In [ ]:
def together_inference(client, query, template, path, choices, condition):
    prompt = template.format(question=query, cond=condition, path=path, options=choices)
    response = client.chat.completions.create(
        # model="meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo",
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
        messages=[{"role": "user", "content": prompt}],
    )

    # print(response.choices[0].message.content)
    return response.choices[0].message.content

In [ ]:
df = pd.read_csv("/home/cc/PHD/HealthBranches/results/results_QUIZ_bench_Llama-3.1-405B-Instruct-Turbo.csv")
df["zero_shot_new"] = df["zero_shot"].apply(extract_option)

big_filtered = df[df["zero_shot_new"].str.upper() != df["real"].str.upper()]

# %%
folder_path = f"{PATH}questions_pro/ultimate_questions_v3_full_balanced.csv"
questions = pd.read_csv(folder_path)

template = CHECK_QUESTION_OPIONS 

rows = []
questions = pd.read_csv(folder_path)
questions = questions[questions["question"].isin(big_filtered["question"])] # Per test su 405B

In [ ]:
print(len(questions))

In [ ]:
with alive_bar(len(questions)) as bar:
    for index, row in questions[:20].iterrows():
        res = []
        try:
            opts = ast.literal_eval(row['options'].replace("['", '["').replace("']", '"]').replace("', '", '", "'))
            
            if not isinstance(opts, list) or len(opts) != 5:
                print(f"Skipping row {index} due to invalid options")
                continue  # Skip this iteration if the condition is not met

        except (ValueError, SyntaxError):
            print(f"Skipping row {index} due to value/syntax error")
            continue  # Skip if there's an issue with evaluation

        txt_name = row['condition'].upper()+".txt"
        txt_folder_name = f"{PATH}data/kgbase-new/"

        try:
            with open(os.path.join(txt_folder_name, txt_name), 'r') as file:
                text = file.readlines()
        except Exception:
            print(os.path.join(txt_folder_name, txt_name))
            print(f"{txt_name} text is EMPTY!")
            continue    
        

        # res.append(together_inference(client, row['question'], template, row['path'], opts, row['condition'].lower()))
        print(row['answer'])
        print(together_inference(client, row['question'], template, row['path'], opts, row['condition'].lower()))
        # res.append(row["correct_option"])
        # res.append(row['question'])
        # res.append(row['path'])
        # res.insert(0, row['condition'].lower())

        # rows.append(res)
        bar()


#     df = pd.DataFrame(rows, columns=["name", "zero_shot", "real", "question", "path"])
#     df.to_csv(f"{PATH}results/results_quiz_405B.csv", index=False)

# print(f"Model BIG done!\n")
